In [1]:
# Load the aiida profile
from aiida import load_profile
aiida_profile = load_profile()
print(f'profil name: {aiida_profile.name}')

## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code, QueryBuilder, WorkChainNode
                       , CalcJobNode)
from aiida.transports import transport
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

profil name: default


In [2]:
# An attemp to run a bunch of wc of the double impurity from two groups of single impurity wc or 
# two list of the single impurity wc

def submit_double_imp_wc_bunch( si_imp_list1, si_imp_list2, kkr_code, kkr_imp_code, builder_options,  
                                succ_group_label, succ_group_descr, offset_imp2={'index':1}, 
                                max_submission=20, setting=None, gf_host_remote=None,
                                scf_wf_parameters=None, params_kkr_overwrite=None, dry_run= True,
                                max_fail_wc = 10, debug= False
                              ):
    import time as t
    """
    params: 
        : Two lists : (si_imp_list1, si_imp_list2) : Two single imp wc list for combination
        : kkr_imp_code : kkr_imp code for kkr_imp_sub_wc
        : kkr_code : kkr_code for kkr_flex_wc
        : Builder_options Dict : computer settings.
        : succ_group_label str : Create a group by this name if it does not exist in the DB.
        : succ_group_descr str : Set the created group description either it exists or not, it not a new group
                            will be created.
        : offset_imp2 : {'index':1}; for 0,1,.. same unit cell or nearest unit cell and so on.
        : max_submissioin : This is the max number of wc for submission runing at the same time.
        : Settings Dict ot dict : Parameter settings.
        : gf_host_remote : RemoteData : Remote kkr flex files
        : scf_wf_parameters : wf_parameters for the scf.wf_parameters
        : params_kkr_overwrite : wf_parameters for host.wf_parameters
        : dry_run : Just for check the code
        : max_fail_wc = stop submission if failed calculation is equal or more than max_fail_wc
        : debug : Also for check the coding
    """    
    # Create group or load existing group
    succ_group = group_not_exist_create(group_label=succ_group_label, group_descr=succ_group_descr)
    fail_group = group_not_exist_create(group_label=succ_group_label+'_fail', group_descr=succ_group_descr+'_fail')
    all_submission_group = group_not_exist_create(group_label=succ_group_label+'_all_submit', group_descr=succ_group_descr+'_all_submit')    
#     test_data and prepare for further
    if not isinstance(si_imp_list1, list):
        print('The given impurity wc list 1 is not the list type.')
        return print('Please provide single_imp1_list.')
    if not isinstance(si_imp_list2, list):
        print('The given impurity wc list 2 is not the list type.')
        return print('Please provide single_imp2_list.')
    
    all_combination_dict = {}
    all_success_dict = {}
    all_submission_dict = {}
    all_resedue_dict = {}
    
    tot_wc_num = 0
    all_submission_num = 0
    all_success_num = 0
    all_failed_num = 0
    
    if isinstance(offset_imp2, dict):
        offset_imp2 = Dict(dict=offset_imp2)
    
    # Here to create all the possible dict
    for i in si_imp_list1[:]:
        for j in si_imp_list2[:]:
            node_truple = (i, j)
            pk_truple = (i.pk, j.pk)
#            imp1_info = i.inputs.impurity_info.get_dict()
#           imp2_info = j.inputs.impurity_info.get_dict()
            offset_index= offset_imp2.get_dict()['index']
#            ilayer1 = str(imp1_info['ilayer_center'])
#            ilayer2 = str(imp2_info['ilayer_center'])
#            label= i.label.split(':')[0] + ':'+ j.label+ '_il_' + ilayer1+'_il_' + ilayer2+'_offset_'+str(offset_index)        
            label= node_label_creation(node1=i, node2=j, offset_index= offset_index)
            all_combination_dict[tot_wc_num] = {'node_truple' : node_truple,
                                                'pk_truple' : pk_truple,
                                                'label' : label,
                                                'submission': None}
            tot_wc_num +=1
    
    all_resedue_dict = all_combination_dict.copy()
    
    # To start the submission process, save the success nodes and failed nodes into the corresponding
    # group
    N=0
    for key, val in all_combination_dict.items():

        imp1_wc_node = all_combination_dict[key]['node_truple'][0]
        imp2_wc_node = all_combination_dict[key]['node_truple'][1]
        label = all_combination_dict[key]['label']
        
## Check the wc exists in the all_submission_group if yes take them into the success of failed group. If still running 'still_running= True' 
        stil_running= False
        wc_pre_submit, pre_submit_wc_pk= check_wc_exist_in_group(all_submission_group, wc_label=label)
        
        if wc_pre_submit:
            pre_submit_node= load_node(pre_submit_wc_pk)
            all_submission_num += 1
            if pre_submit_node.is_finished == True:
                if pre_submit_node.exit_status == 0 : 
                    all_success_num +=1
                    print(f'Node pk: {pre_submit_wc_pk} is already submited and successfully finished. Adding to the group: {succ_group}') 

                    succ_group.add_nodes(pre_submit_node)
                    all_submission_group.remove_nodes(pre_submit_node)

                    del_node(node_pks=[pre_submit_node.pk], dry_run=False, verbosity=0, debug=False, only_remote_dir=True)
                else:
                    all_failed_num += 1
                    print(f'Node pk: {pre_submit_wc_pk} is already submited and failed. Adding to the group: {fail_group}')
                    fail_group.add_nodes(pre_submit_node)
                    all_submission_group.remove_nodes(pre_submit_node)

            elif pre_submit_node.is_excepted:
                all_failed_num += 1
                print(f'Node pk: {pre_submit_wc_pk} is already submited and failed. Adding to the group: {fail_group}')
                fail_group.add_nodes(pre_submit_node)
                all_submission_group.remove_nodes(pre_submit_node)
    
## ---------------------------        
        
        wc_pre_exist, _ = check_wc_exist_in_group(succ_group, wc_label=label)
        wc_pre_fail, _ = check_wc_exist_in_group(fail_group, wc_label=label)
 
        
        
                
        
        if wc_pre_exist or wc_pre_fail or wc_pre_submit:
#             print('Already one wc named as {} is exist in this group'.format(label))
            N += 1
            # To reduce the tot_wc_num as they are either failed or sucessfully finished earlier
            tot_wc_num -= 1
            continue
        else:
            print('val : ', val)
        
        if all_failed_num == max_fail_wc:
            print('This is for loop break, as all_failed_num == max_fail_wc')
            break
        
        if debug: 
            print('imp1_pk for submission', imp1_wc_node.pk)
            print('imp2_pk for submission', imp2_wc_node.pk)


        if not dry_run: 
            submission = submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, offset_imp2,
                                            options, settings, dry_run, label, gf_host_remote,
                                            scf_wf_parameters, params_kkr_overwrite )
            t.sleep(10)
            print(f"THe submitted combine Imps is : {submission.pk}")
            all_submission_dict[key] = all_combination_dict[key].copy()
            all_submission_dict[key]['submission'] = submission
            all_submission_num += 1
            all_submission_group.add_nodes(submission)
        while((all_submission_num - all_success_num - all_failed_num >= max_submission) or 
                (tot_wc_num - all_submission_num == 0) or (all_failed_num == max_fail_wc)):
                t.sleep(60*2)
                
#----------------------------------------------------------------------------------

                pop_list = []
                for submit_key in all_submission_dict.keys():
                    submission = all_submission_dict[submit_key]['submission']
                    if submission.is_finished == True:
                        if submission.exit_status == 0 : 
                            all_success_num +=1
                            print(f'all_success_num : {all_success_num}, and last pk : {submission.pk}') 
                            all_success_dict[submit_key] = all_combination_dict[submit_key].copy()
                            all_resedue_dict.pop(submit_key)
                        
                            succ_group.add_nodes(submission)
                            all_submission_group.remove_nodes(submission)
                            pop_list.append(submit_key)
                            
                            del_node(node_pks=[submission.pk], dry_run=False, verbosity=0, debug=False, only_remote_dir=True)
                        else:
                            all_failed_num += 1
                            print(f'all_failed_num : {all_failed_num}, last pk : {submission.pk}')
                            fail_group.add_nodes(submission)
                            all_submission_group.remove_nodes(submission)
                            pop_list.append(submit_key)
                            
                    elif submission.is_excepted:
                        all_failed_num += 1
                        print(f'all_excepted_num : {all_failed_num}, last pk : {submission.pk}')
                        fail_group.add_nodes(submission)
                        all_submission_group.remove_nodes(submission)
                        
                        pop_list.append(submit_key)
                        
#---------------------------------------------------------------------------------
                garbage = [all_submission_dict.pop(pop_key) for pop_key in pop_list[:]]
                    
                if len(all_submission_dict) == 0 :
                    print('This is while loop break, as all_submission_dict.')
                    break
                if all_failed_num == max_fail_wc :
                    if (all_submission_num - all_success_num - all_failed_num) == 0 :
                        print('This while loop break, as (all_submission_num - all_success_num - all_failed_num) == 0')
                        break
                
    print('total failed or sucsess wc : ', N)
#     return all_success_dict, all_resedue_dict

def submit_combine_imp(imp1_wc_node, imp2_wc_node, kkr_code, kkr_imp_code, offset_imp2, options, 
                       settings, dry_run, label, gf_host_remote=None,
                       scf_wf_parameters = None, params_kkr_overwrite=None):
    
    from aiida_kkr.workflows import combine_imps_wc, kkr_imp_sub_wc, kkr_flex_wc
    from aiida.orm import Dict
    imp1_output = imp1_wc_node.outputs.workflow_info
    imp2_output = imp2_wc_node.outputs.workflow_info
    if scf_wf_parameters==None:
        sub_wc1_node = imp1_wc_node.get_outgoing(node_class=kkr_imp_sub_wc).first().node
        scf_wf_parameters = sub_wc1_node.inputs.wf_parameters
    
    if gf_host_remote==None:
        if params_kkr_overwrite==None:
            ##TODO: In case the gf_remote is from the parent calc: Another possible way to do it as, from inputs remote_gf->kkr_flex_wc 
            sub_gf_write_out = imp1_wc_node.get_outgoing(node_class=kkr_flex_wc).first().node
                
            params_kkr_overwrite = sub_gf_write_out.inputs.params_kkr_overwrite
    if settings==None:
        settings = imp1_wc_node.inputs.wf_parameters_overwrite
    if label==None:
        label = 'pk' + str(imp1_wc_node.pk)+':'+ str(imp1_wc_node.pk)
    
    builder = combine_imps_wc.get_builder()
    builder.impurity1_output_node = imp1_output
    builder.impurity2_output_node = imp2_output
    if isinstance(offset_imp2, dict):
        builder.offset_imp2 = Dict(dict=offset_imp2)
    else:
        builder.offset_imp2 = offset_imp2
    
    builder.scf.kkrimp = kkr_imp_code
    builder.scf.options = options
    builder.scf.wf_parameters = scf_wf_parameters
    
    if gf_host_remote==None:
        builder.host_gf.kkr = kkr_code
        builder.host_gf.options = options
        builder.host_gf.params_kkr_overwrite = params_kkr_overwrite #host_gf.inputs.wf_parameters
    else:
        builder.gf_host_remote = gf_host_remote
    if settings!=None:
        if isinstance(settings, dict):
            builder.wf_parameters_overwrite = Dict(dict=settings)
        else:
            builder.wf_parameters_overwrite = settings
            
    builder.metadata.label = label
    if not dry_run:
        submission = submit(builder)
        return submission
    else:
        msg = ' This is dry_run. '
        return msg
    
# This function create perfect node label by combining two single-single workflow 
# or for two sigle-double workflow. The first two inputs node1 and node2 are aiida nodes 
# and offset_index is integer 
def node_label_creation(node1, node2, offset_index):
    OtherLabel1= node1.label
    OtherLabel2= node2.label
    ilSeparator= '_il_'
    offSeparator= '_Off_'

    ## To collect the host label
    LabComs1= OtherLabel1.split(ilSeparator)
    LabComs1= LabComs1[0].split(':')
    host_label= LabComs1[-1]
    ## Combining imps
    LabComs1= OtherLabel1.split(host_label)
    LabComs2= OtherLabel2.split(host_label)
    Combine_imps= LabComs1[0]+LabComs2[0]
    ## ilayers
    LabComs1= OtherLabel1.split(ilSeparator)
    LabComs2= OtherLabel2.split(ilSeparator)
    Combine_il= LabComs1[1].split(offSeparator)[0] + '_' + LabComs2[1].split(offSeparator)[0]
    ## offset settings part part
    if offSeparator in OtherLabel1:
        LabComs1= OtherLabel1.split(offSeparator)
        LabComs1= ''+LabComs1[-1]
    else:
        LabComs1= ''
    if offSeparator in OtherLabel2:
        LabComs2= OtherLabel2.split(offSeparator)
        LabComs2= LabComs2[-1].split('_')
        LabComs2= [ str(int(i)+offset_index) for i in LabComs2]
        rubish= ''
        for i in LabComs2:
            rubish += (i + '_')
        rubish += 'RemovingTheLastHypeh'
        rubish= rubish.replace('_RemovingTheLastHypeh','') 
        print(rubish)
        LabComs2= str((0+offset_index))+ '_' +rubish 
    else:
        LabComs2= str(0+offset_index)
    if LabComs1 != '':
        Combine_off= LabComs1 + '_'+ LabComs2
    elif LabComs1 == '':
        Combine_off= LabComs2
    CombineLabel= Combine_imps+ host_label + ilSeparator + Combine_il + offSeparator + Combine_off
#    print(f'New Label: {CombineLabel}, from the old labels {node1.label} and {node2.label}')
    return CombineLabel
        
def group_not_exist_create(group_label, group_descr=None):
    from aiida.orm import load_group, Group
    """
        Check the group exist either must create
    """
    if group_descr==None:
        group_descr='No Description is added'
    
    try:
         group = load_group(group_label)
    except:
        print('Group named {} is not exist but is being created .'.format(group_label))
        group = Group(label=group_label, description=group_descr)
        group.store()
        print('Newly created group pk {}'.format(group.pk))
    return group


def check_wc_exist_in_group(group, wc_label=None, wc_pk=None):
    nodes_list = list(group.nodes)
    nodes_label = [i.label for i in nodes_list[:]]
    nodes_pk = [i.pk for i in nodes_list[:]]
    
    if wc_label in nodes_label:
        print('node_label-{} is exist in the group-{}.'.format(wc_label, group.label))
        index= nodes_label.index(wc_label)
        wc_pk= nodes_pk[index]
        return True, wc_pk
              
    elif wc_pk in nodes_pk:
        print('node_pk-{} is exist in the group-{}.'.format(wc_label, group.label))
        return True, wc_pk
              
    else:
        return False, None
    
    

#### THIS FUNCTION IS SUCCESSFULLY TESTED ####

## Trying to develop to delete the node from database
## and at the same time from the remote folder

## Add the following possible services
# 1. Give the print option for how many decendant node will be 
#       and take the permission. Add option wether need to take 
#         permission or not.
# 2. Also print how many calcjob node will be deleted under permission 
# 3. Print'remote directory' to check that all the data from the
#     remote dir are deleted or not.
## Technique
# 1. Use the QuaryDB()
# Use the cleandir



#Section-1: delete node from the database as well as remote work directory
### A function to delete the data of calculation output of calcjob list (pks) from the remote dir.

# later add it with the del_node  function
# This is successfully done

# please note that it is tested for one calc list
def delete_remote_workdir(pks, verbosity=0, dry_run= True):
    from aiida.common import exceptions, NotExistent
    from aiida.orm import load_node
    from aiida.orm import computers
    import sys
    # TODO : add the verbosity as discused here 
    # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

    """
    :param pks: calc node list
    
    """
    removed_path_list = [] # The part of the path will be deleted
    remote_path_list = []  # The original path
    updated_path_list = [] # After removing the part of the path
    loadable_list = [] # To load the node and save it loadable_list
    loaded_node_list = []
    # To check the loadable calcjob list
    for pk in pks:
        try:
            loaded_node = load_node(pk)
        except exceptions.NotExistent:
            print('This is calcjob node'.format(pk))
            loaded_node = pk
            loaded_node_list.append(loaded_node)
            #             sys.exit()
        else:
            loaded_node_list.append(loaded_node)
    # Computer data
    
   
    for node in loaded_node_list:
        load_pk = node
        # computer data
        computer = load_pk.computer
        computer_name = computer.label
        print(computer_name)
        try:
            remote_path = load_pk.get_remote_workdir()
            
        except NotExistent as ex:
            print(f'Node (label,pk) ({node.label, node.pk}) does not have remote workdir. with exception : {ex}')
        if isinstance(remote_path, str):
            remote_path_list.append(remote_path)
            delete_folder = remote_path.split('/')[-1]
            removed_path_list.append(delete_folder)

            new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
            updated_path_list.append(new_remote_path)
        
        
    if dry_run or verbosity == 3:

        for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
            print('Before the delation the original path list : {}\n'.format(paths[0]))
            print('After deletion the modefied or updated path : {}'.format(paths[1]))
    if verbosity == 3 or verbosity == 2:
        val = input("Are you agree to clean the remote workdir (y/n) : ")
    else:
        val = 'y'
    if str(val)=='y' or str(val)=='Y':
        if not dry_run:
            for remote_path in remote_path_list:
                try:
                    # Open the connection to the remote folder/dir via transport
                    computer_transport = computer.get_transport()
                    is_transport_open = computer_transport.is_open
                    if not is_transport_open:
                        computer_transport.open()
                    computer_transport.rmtree(remote_path)
                except IOError as ex:
                    print('Uable to open the computer transport: ', ex)
    else:
        print('Nothing to clean from the remote workdir!')
# section-5:delete node from the database as well as remote work directory
## It returns all the calcjob from a WC node
def find_calcJob(pk_or_node, debug=True):
    
    calcjob_node_list=[]
    wc_node_list = []
    try:
        if isinstance( pk_or_node, int):
            if debug:
                print('This is pk')
            node = load_node(pk_or_node)
        else:
            if debug:
                print('This is node.')
            node= pk_or_node
    except:
        print('{} is nither node ID nor aiida_node. '.format(pk_or_node))
        
    ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
    calc_list, wc_list = get_calcjob_wc(node)
    calcjob_node_list += calc_list
    
    while len(wc_list)!=0:
        new_wc_list = []

        for i in wc_list[:]:
            calc_list, wc_list = get_calcjob_wc(i)
            new_wc_list += wc_list
            calcjob_node_list += calc_list
            
        wc_list = new_wc_list

    return calcjob_node_list

## This function returns calcjob_list and wc_list from a wc or calcjob node   
def get_calcjob_wc(node):
    """
    :param: node
    :return: workchain node list and calcjob node list
    """ 
    from aiida.orm import CalcJobNode, WorkChainNode
    wc = []
    calc_job = []
    
    if node.node_type == 'process.workflow.workchain.WorkChainNode.':
        
    # here all outgoing worchain node
        out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
        wc = [i.node for i in out_going_wc[:]]
        
    # here all outgoing calcjob node
        out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
        calc_job = [i.node for i in out_going_calc[:]]
                    
    elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
        calc_job.append(node)
    
    return calc_job, wc

# This is the final del_node_function. Using this function for any specific wc node the node from the 
# Db as well as the calcjob data from the remote workdir can be deleted.
def del_node(node_pks, dry_run=True, verbosity=3, debug=True, only_remote_dir=False,
            only_database=False):
    """
    1. This function will delete the node data from the database and also from the remote_dir
    
    :params node_pks: (list) list of workchain to delete from database as well as from remote workdir
    :param verbosity: 0 prints nothing.  This is for workdir and wc
                      1 prints just sums and total.   This is for workdir but not for wc
                      2 prints indivisual nodes.  This is for workdir and wc
    :param dry_run: Do not delete anything just show the status as in the verbosity given
    """
    from aiida.orm import load_node
    from aiida.manage.database.delete.nodes import delete_nodes
    
        
    calcjobs_list = []
    pks_given = []
    for i in node_pks:    
        try:
            if isinstance( i, int):
                if debug:
                    print('This might be pk or uiid')
                node = load_node(i)
            else:
                if debug:
                    print('This might be a node.')
                node= i
        except:
            print('{} is nither node ID nor aiida_node. '.format(i))
        
        pks_given.append(node.pk)
        
        calcjobs = find_calcJob(node, debug)
        calcjobs_list += calcjobs
        print('calcjob list : ', calcjobs_list,)
    if only_remote_dir:
        delete_remote_workdir(calcjobs_list, verbosity=verbosity, dry_run=dry_run)
    if only_database:
        delete_nodes(pks_given, verbosity=verbosity, dry_run=dry_run,force=False)
    

In [3]:
# One very frequently used option
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }
oscar_matadata = {'max_wallclock_seconds': 8*60*60,
                  'resources':{'tot_num_mpiprocs': 12, 
                  'num_machines': 4},
                  'custom_scheduler_commands': '#SBATCH -p oscar --nodes=1 --ntasks=12\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
                  'withmpi':True
                 }

oscar_matadata = {'withmpi': True,
         'resources': {'num_machines': 1, 'tot_num_mpiprocs': 12},
         'queue_name': 'oscar',
         'max_wallclock_seconds': 86400}

voro_code = Code.get_from_string('voro@claix18_init')
kkr_code = Code.get_from_string('kkr@claix18_init')
kkrimp_code = Code.get_from_string('kkrflex@claix18_init')
iffslurm_voro = Code.get_from_string('voro@iffslurm')
iffslurm_kkr= Code.get_from_string('kkr@iffslurm')
iffslurm_kkrimp= Code.get_from_string('kkrflex@iffslurm')

In [4]:
# Note that here for doped 3d and 4d atoms into the Bi2Te3 host saved in group-74

group_74 = load_group(74)
## first node of group-74 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

# gf_host_remote = imp_wc_with_gf.inputs.remote_data_gf
# host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [5]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RCLUSTZ'] *= 1
kkr_flex_kkr_param['NSHELD'] = 2500

NameError: name 'kkr_calc' is not defined

In [6]:
#kkr_flex_kkr_param

In [7]:
settings = combine_imps_wc.get_wf_defaults()
builder = combine_imps_wc.get_builder()
settings['jij_run']= True
settings['retrieve_kkrflex'] = False
settings['mag_init']= False
settings['threshold_switch_high_accuracy'] = 1e-2
settings['convergence_criterion'] = 1e-7 #convergence_criterion
settings['threshold_aggressive_mixing']= 0.05
settings['strmix']= 0.01
settings['aggrmix'] = 0.05
settings['broyden-number'] = 20
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

Version of workflow: 0.3.1


In [8]:
options = scf_node.inputs.options.get_dict()
options['max_wallclock_seconds'] = 36000*1
# options

NameError: name 'scf_node' is not defined

In [9]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
builder.scf.kkrimp = scf_node.inputs.kkrimp
builder.scf.options = Dict(dict=options)
builder.scf.wf_parameters = scf_node.inputs.wf_parameters

# host_gf namespace setup
builder.host_gf.kkr = host_gf.inputs.kkr
builder.host_gf.options = host_gf.inputs.options
builder.host_gf.params_kkr_overwrite = Dict(dict=kkr_flex_kkr_param) #host_gf.inputs.wf_parameters

builder.wf_parameters_overwrite = Dict(dict=settings)
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

NameError: name 'scf_node' is not defined

In [10]:
# To submit the builder
# combined_imp_submission = submit(builder)

In [11]:
# Note that here for doped 3d atoms into the Bi2Te3 host

##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# To run 1 to 1
group_74 = load_group(74) # ilayer-3
group_91 = load_group(91) # ilayer-4

## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp1_wc = group_74.nodes[0]
imp2_wc = group_91.nodes[0]
# To run 1 to 1
## %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%



scf = imp1_wc

host_kkr = Code().get_from_string('kkr@claix18_init')

scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp1_wc.outputs.workflow_info
imp2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':2}

## For kkr_writeout_step: parameters for kkr calculation
params_kkr_overwrite = {'BZDIVIDE': [80,80,80],
                        'strmix': 0.001,
                        'NSHELD': 2500, 
                        'KPOIBZ': 1000000 # To increase the KPOIBZ(NxNyNz= 100*100*100)
                       }
params_kkr_overwrite= Dict(dict=params_kkr_overwrite)


settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = True
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.01
settings['aggrmix'] = 0.05
settings['threshold_aggressive_mixing'] = 0.01
settings['convergence_criterion'] = 1e-8
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = True
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10
settings['broyden-number'] = 20
##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#This part for runing
# if this function works then delete the below section
ilayer1 = imp1_wc.inputs.impurity_info.get_dict()['ilayer_center']
ilayer2 = imp2_wc.inputs.impurity_info.get_dict()['ilayer_center']
offset_index= offset_imp2['index']

label = imp1_wc.label.split(':')[0]+':'+imp2_wc.label+'_il_'+str(ilayer1)+'_il_'+str(ilayer2)+'_offset_'+str(offset_index)
print('label : ',label)
#submit_combine_imp(imp1_wc_node=imp1_wc, imp2_wc_node=imp2_wc, kkr_code=host_kkr, kkr_imp_code=scf_kkrimp, offset_imp2=offset_imp2, 
#                    options = options, scf_wf_parameters=None, params_kkr_overwrite=params_kkr_overwrite,settings=settings, 
#                    gf_host_remote=None, dry_run=False, label=label
#                   )



## Delete This node as they are after the saving it into the grpup
# with retrieve kkrflex file: False; 66160
# with retrieve kkrflex file: True; 66204


Version of workflow: 0.3.1
label :  Sc:Sc:Bi2Te3_il_4_il_3_il_4_offset_2


In [9]:
## Here trying to work with host Au and other impurity
## THe calc is taken from the wasmer
imp_Au_group = load_group(85)
wc_list = list(imp_Au_group.nodes)

# This is the single imp wc
imp_wc_1 = wc_list[0]
# This is the double imps wc
#imp_wc_1 = load_node(102808)
print('imp_wc_1 label:', imp_wc_1.label)
#print('imp_info_dict:', imp_wc_1.inputs.impurity_info.get_dict())
# print('first imp wc : ', imp_wc_1.label)
imp_wc_2 = wc_list[1]
print('imp_wc_2 label:', imp_wc_1.label)
# print('econd imp wc : ', imp_wc_2.label)
# setting the scf and host namespace
scf = imp_wc_2

params_kkr_overwrite = Dict(dict={'NSHELD' : 2500,
                               })

kkr_claix18 = Code().get_from_string('kkr@claix18_init')
kkrimp_claix18 = Code().get_from_string('kkrflex@claix18_init')
voro_claix18 = Code().get_from_string('voro@claix18_init')

options = Dict(dict=metadata_option_1)

imp1_output_node = imp_wc_1.outputs.workflow_info
imp2_output_node = imp_wc_2.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.001
settings['aggrmix'] = 0.06
settings['threshold_aggressive_mixing'] = 0.06
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = kkrimp_claix18 #iffslurm_kkrimp
combine_imp_builder.scf.options = Dict(dict = metadata_option_1)
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters

# combine_imp_builder.gf_host_remote
# host_gf namespace setup
combine_imp_builder.host_gf.kkr = kkr_claix18
combine_imp_builder.host_gf.options = Dict(dict=metadata_option_1)
combine_imp_builder.host_gf.params_kkr_overwrite = params_kkr_overwrite #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = imp_wc_1.label + imp_wc_2.label#node_label_creation(imp_wc_1, imp_wc_2,offset_index=offset_imp2)
submission = submit(combine_imp_builder)

imp_wc_1 label: Ru:Au
imp_wc_2 label: Ru:Au
Version of workflow: 0.3.1


combine

In [17]:
group_74 = load_group(74)
group_91 = load_group(91)
group_74 = list(group_74.nodes)
group_91 = list(group_91.nodes)
# 3vs3 indx1
# 3vs4 indx1
###Code list
kkr_code = Code.get_from_string('kkr@claix18_init')

In [21]:
# Prepartion to submit il-3 Vs il-4 for the 3d_&_4d block
## To run  il-3 Vs il-4 take the code from 'Some_code_snipet.ipynb'

# extract from parent wc
## node 24418 is succefull wc for li-3 il-3 from node 16215 and 16237
## node 24493 is succefull wc for li-4 il-4 from node 18853 and 18853
## node 24510 is succefull wc for li-4 il-4 from node 18853 and 16215

#### This part from group 74 and 91 ----------------------------
## To run the il 3 vs 4 please run from this pk: 67687 but before test the label
parent_wc = load_node(67687)
## This part for the failed combined calculations by increasing the BZDIVIDE-------------
params_kkr_overwrite = {'BZDIVIDE': [80,80,80],
                        'strmix': 0.02,
                        'NSHELD': 2500, 
                        'KPOIBZ': 1000000 # To increase the KPOIBZ(NxNyNz= 100*100*100)
                       }
## ------------------------------------------

kkr_imp_code = parent_wc.inputs.scf__kkrimp
options = Dict(dict=metadata_option_1)
scf_wf_parameters = parent_wc.inputs.scf__wf_parameters

settings = parent_wc.inputs.wf_parameters_overwrite
## If need use the customised settings
settings_customised = {}
settings_customised['jij_run'] = True
settings_customised['lmdos'] = False
settings_customised['strmix'] = 0.02
settings_customised['aggrmix'] = 0.008
settings_customised['threshold_aggressive_mixing'] = 0.01  ## it takes the first scf limit
settings_customised['convergence_criterion'] = 1e-8
settings_customised['threshold_switch_high_accuracy'] = 1e-2
settings_customised['retrieve_kkrflex'] = True
settings_customised['mag_init'] = False
settings_customised['mixreduce'] = 0.5
settings_customised['nsteps'] = 120
settings_customised['kkr_runmax'] = 10

gf_host_remote = parent_wc.outputs.remote_data_gf

offset_imp2 = parent_wc.inputs.offset_imp2

### In case to change the offset_number
offset_imp2= offset_imp2.get_dict()
offset_imp2['index']= 2
offset_imp2= Dict(dict=offset_imp2)
print(offset_imp2.get_dict())
### end here to change the offset_imp2

si_imp_list1 = group_74
si_imp_list2 = group_91

succ_group_label = '3d_4d_dope_Bi2Te3_il_3:il_4_offset_2'
succ_group_descr = 'only from group ID 74 and 91'

#submit_double_imp_wc_bunch( si_imp_list1= si_imp_list1, 
                             si_imp_list2= si_imp_list2, 
                             kkr_code=None, #kkr_code, 
                             kkr_imp_code= kkr_imp_code, 
                             builder_options= options,  
                             succ_group_label= succ_group_label,
                             succ_group_descr= succ_group_descr,
                             offset_imp2= offset_imp2,
                             max_submission= 50,
                             setting = Dict(dict=settings_customised), 
                             gf_host_remote= gf_host_remote,
                             scf_wf_parameters= scf_wf_parameters, 
                             params_kkr_overwrite= Dict(dict=params_kkr_overwrite),
                             dry_run= False,
                             max_fail_wc= 5,
                             debug= False
                               )
## 
#### This part from group 74 and 91 ----------------------------

{'index': 2}
node_label-Sc:Sc:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Mn:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Cr:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:V:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Ti:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Ni:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Fe:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Y:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Zn:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Zr:Bi2Te3_il_3_4_Off_2 is exist in the group-3d_4d_dope_Bi2Te3_il_3:il_4_offset_2.
node_label-Sc:Cu:Bi

In [20]:
# Prepartion to submit il-3 Vs il-3 for the 3d_&_4d block
## To run  il-3 Vs il-3 take the code from 'Some_code_snipet.ipynb'

# extract from parent wc
## node 24418 is succefull wc for li-3 il-3 from node 16215 and 16237
## node 24493 is succefull wc for li-4 il-4 from node 18853 and 18853
## node 24510 is succefull wc for li-4 il-4 from node 18853 and 16215

##This part for not successful wc to retry them------
params_kkr_overwrite = {'BZDIVIDE': [80,80,80],
                        'strmix': 0.01,
                        'NSHELD': 2500,
                        'KPOIBZ': 1000000 ## To increase the KPOIBZ: NxNyNz=100 100 100
                       }
##---------------------------------------------------
parent_wc = load_node(67670)

# To run the il_3_3 for offset index 2 try with pk: 67670 is it is successful and also before starting please checck the label

#### This part from group 74 and 91 ----------------------------

settings_customised = {}
# settings_customised['RCLUSTZ'] = 2.3
settings_customised['jij_run'] = True
settings_customised['strmix'] = 0.001
settings_customised['aggrmix'] = 0.005 ## Maybe it takes the first scf limit
# settings_customised['threshold_aggressive_mixing'] = 0.02
settings_customised['convergence_criterion'] = 1e-8
settings_customised['threshold_switch_high_accuracy'] = 1e-2
settings_customised['retrieve_kkrflex'] = False
settings_customised['mag_init'] = False
settings_customised['mixreduce'] = 0.2
settings_customised['nsteps'] = 100
settings_customised['kkr_runmax'] = 10



kkr_imp_code = parent_wc.inputs.scf__kkrimp
options = parent_wc.inputs.scf__options
scf_wf_parameters = parent_wc.inputs.scf__wf_parameters

settings = parent_wc.inputs.wf_parameters_overwrite

gf_host_remote = parent_wc.outputs.remote_data_gf
# params_kkr_overwrite = parent_wc.

offset_imp2 = parent_wc.inputs.offset_imp2
### In case to change the offset_number
offset_imp2 = offset_imp2.get_dict()
offset_imp2['index']= 2
offset_imp2 = Dict(dict=offset_imp2)
print(offset_imp2.get_dict())
### end here to change the offset_imp2

si_imp_list1 = group_74
si_imp_list2 = group_74

succ_group_label = '3d_4d_dope_Bi2Te3_il_3:il_3_offset_2'
succ_group_descr = 'only from group ID 74 and 74'

#submit_double_imp_wc_bunch( si_imp_list1= si_imp_list1, 
                            si_imp_list2= si_imp_list2, 
                            kkr_code= None, #kkr_code, 
                            kkr_imp_code= kkr_imp_code, 
                            builder_options= options,  
                            succ_group_label= succ_group_label,
                            succ_group_descr= succ_group_descr,
                            offset_imp2= offset_imp2,
                            max_submission= 50,
                            setting= settings_customised, 
                            gf_host_remote= gf_host_remote,
                            scf_wf_parameters= scf_wf_parameters, 
                            params_kkr_overwrite= Dict(dict=params_kkr_overwrite),
                            dry_run= False,
                            max_fail_wc= 1,
                            debug= False
                        )
#first combine calc=67634(F, Del), 67653(checck, Del)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 77)

In [ ]:
# failed and delete the combined_imps_wf: 65407
# Runing combined imps wf: 65235, 65296, 65423, 65486


In [ ]:
# delete all the node in 106
group_106= load_group(122)# 73
nodes= list(group_106.nodes)

# DElete only remote data:102222

#del_node(node_pks=nodes, dry_run=False, verbosity=3, debug=True, only_remote_dir=True, only_database=False)

In [ ]:
transport.Transport.is_open

In [ ]:
qb1 = QueryBuilder()
Qb_node_list = qb1.append(WorkChainNode,
          #filters={
#               'and':[
#                   {'attributes.process_label':'kkr_imp_wc'},
#                   {'attributes.exit_status':{'!in':[0]}}
                  
#               ],
             # 'id':{'>':57959},
 #             'attributes.process_label':'kkr_imp_wc',
#
              
 #         }
         ).all()

In [ ]:
nodes_list = [node[0] for node in Qb_node_list]
#for node in Qb_nodes_list:
#    print(f'node label: {node.label}, node pk: {node.pk}, exit status: {node.exit_status}')

len(nodes_list)

In [ ]:
group_112= load_group(116)
group_nodes_list= list(group_112.nodes)

#del_node(node_pks=group_nodes_list, dry_run=False, verbosity=3, debug=True, only_remote_dir=True,
#           only_database=True)